# Système de recommandation

In [1]:
import math
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import pairwise_distances
from sklearn.neighbors import NearestNeighbors

## 1.Chargement des données

In [2]:
df_ratings = pd.read_csv(
    'Reviews.csv',
    usecols=['customer', 'productId', 'rating'],
    dtype={'customer': 'int32', 'productId': 'int32', 'rating': 'float32'})

df_products = pd.read_csv(
    'Products.csv',
    usecols=['productId', 'title'],
    dtype={'productId': 'int32', 'title': 'str'})

## 2. action sur les données 'Products'

In [3]:
df_products.shape

(549, 2)

In [4]:
df_products.head()

,productId,title
0,620,Toddler Boys Camo Print Hoodie & Sweatpants
1,619,SHEIN Girls Solid Cami Top & Belted Pants Set
2,618,SHEIN Girls Colorblock Slogan Graphic Pullover
3,617,SHEIN Girls Drop Shoulder Plaid Sweatshirt
4,616,SHEIN Girls Drop Shoulder Letter Hoodie & Camo...


In [5]:
# obtenir le titre de produit a prtir de son Id
def productTitle(productId):
    for ligne in df_products.values:
        if ligne[0]==productId:
            return ligne[1]
    return "Id que vous avez entré ne correspond à aucun produit"

In [6]:
print(productTitle(126))

Boys Coats+T-shirt+Pants


## 3. action sur les données 'Ratings'

In [7]:
df_ratings.shape

(1781, 3)

In [8]:
df_ratings.head()

,customer,productId,rating
0,1,541,4.0
1,1,540,4.0
2,1,539,5.0
3,1,538,3.0
4,1,537,3.0


In [9]:
# obtenir rating d'un produit par un utilisateur aprtir de ProductId et UserId
def productRatByUser(UserId, productId):
    for ligne in df_ratings.values:
        if ligne[0]==UserId and ligne[1]==productId:
            return ligne[2]

In [10]:
productRatByUser(1,541)

4.0

In [11]:
# matrice product_user
product_user_mat = df_ratings.pivot(index='productId', columns='customer', values='rating').fillna(0)
product_user_mat.T.head()       # Transposé du matrice product_user       

productId,48,49,50,51,52,53,54,55,56,57,...,611,612,613,614,615,616,617,618,619,620
customer,,,,,,,,,,,,,,,,,,,,,
1,1.0,2.0,2.0,4.0,1.0,4.0,3.0,2.0,4.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,4.0,1.0,4.0,4.0,4.0,1.0,4.0,1.0,5.0
3,4.0,2.0,4.0,3.0,5.0,4.0,2.0,3.0,3.0,4.0,...,4.0,2.0,2.0,3.0,5.0,5.0,5.0,1.0,5.0,2.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# identifier les produits évalué par chaque utilisateur
rate = {}
ligne_idx ={}
for idUser, ligne in product_user_mat.T.iterrows():
    ligneIdx = [x for x in range(0, len(product_user_mat.T.columns))]  #[0,1,.....] pour 
    combine_PrdId_Rat_idx = list(zip(ligne.index, ligne.values, ligneIdx))   # [(productid, rating, index),....]
    rated = [(prdId, idx) for prdId, rat, idx in combine_PrdId_Rat_idx if rat !=0 ]
    ligne_prdTitleReted = [r[0] for r in rated]
    ligne_index = [r[1] for r in rated]
    rate[idUser] = ligne_prdTitleReted
    ligne_idx[idUser] = ligne_index

In [13]:
# liste des produits évalué par l'utilisateur i
i=1
for prdId in rate[i]:
    print(productTitle(prdId))

Sweater Dress Winter
Beach Dress Woman
Long Party Dresses
new women's hollow slim dress
Midi Dresses Spring Women
Long beaut Dress Elegant
white lace dress women
Autumn New Dresses Classic
Sweater Long Dresses
Cute Large Dress Big Robe
Midi Dress
Long Dress
Midi Dress Women
pregnant t shirt dress
Short Sleeve A Line Dress For Women
Comfortable Party Dress
Long Dresses
White Midi Dress
Casual Dress Female
Black Casual Cartoon Dress
Dress 2020 New Office Lady
mid-length pullover knitted dress
Shirt Dresses Fashion
Print Patchwork Loose Dress
Floral Print Women Dress
Bodycon gothic Dresses
office work dresses
Winter Bodycon Vestidos
ladies work dress 2020
Women Casual Dress
Women Fashion Round Glasses
Women Cat Eye Sun Glasses
Sun Glasses For Women
Women Warm Cotton
Jacket Brand Men Military
New Jackets Parka
Jeans Jacket
Bomber Jacket
Single Breasted Jeans Jacket
Autumn Jacket Men
Jacket Men Fashion
Denim jacket
Casual Jacket Men
Hot Slim Mens Jackets
Female Travel Fashion Hand Bag
Shoul

In [14]:
# identifier les produits qui ne sont pas évalué par chaque utilisateur
norate = {}
ligne_idx2 ={}
for idUser, ligne in product_user_mat.T.iterrows():
    ligneIdx = [x for x in range(0, len(product_user_mat.T.columns))]  #[0,1,.....] pour 
    combine_PrdId_Rat_idx = list(zip(ligne.index, ligne.values, ligneIdx))   # [(productid, rating, index),....]
    rated = [(prdId, idx) for prdId, rat, idx in combine_PrdId_Rat_idx if rat ==0 ]
    ligne_prdTitleReted = [r[0] for r in rated]
    ligne_index = [r[1] for r in rated]
    norate[idUser] = ligne_prdTitleReted
    ligne_idx2[idUser] = ligne_index

In [15]:
# liste des produits non évalué par l'utilisateur i
i=1
for prdId in norate[i]:
    print(productTitle(prdId))     

Men Halloween Pumpkin Design Stud Earrings
Men Halloween Skull Charm Necklace
Men Bat Decor Stud Earrings
Men Spider Design Earring Jackets
Men Rhinestone Decor Skull Earring Jackets
2pcs Men Eye Decor Ring
Men Halloween Pumpkin Design Stud Earrings
Men Rhinestone Decor Skull Design Ring
2pcs Men Eye Design Ring
Men Skull Decor Mittens Bracelet
Men Skull Decor Bracelet
Men Skull Decor Bracelet
Men Hollow Out Metal Decor Necklace
Men Halloween Skull Decor Bracelet
Men Seahorse Decor Earrings
Men Metal Decor Layered Bracelet
Men Snake Design Stud Earrings
Men Gemstone Decor Ring
4pairs Men Geo Design Earrings
Men Scorpion Decor Bracelet
3pcs Men Beaded Bracelet
Men Simple Metal Bracelet
Men Metal Layered Bracelet
Men Round Charm String Bracelet
3pairs Men Simple Double Stud Earrings
Men Serpentine Ring
5pcs Men Heart Decor Nose Ring And One Time Use Disposable Nose Piercing Tool
Men Rhinestone Decor Broken Heart Charm Necklace
3pcs Men Lock Decor Chain Bracelet
4pcs Men Simple Nose Ring


In [16]:
product_user_mat.head()

customer,1,2,3,4,6,8
productId,,,,,,
48,1.0,0.0,4.0,0.0,0.0,1.0
49,2.0,0.0,2.0,0.0,0.0,3.0
50,2.0,0.0,4.0,0.0,0.0,5.0
51,4.0,0.0,3.0,0.0,0.0,3.0
52,1.0,0.0,5.0,0.0,0.0,2.0


In [17]:
# obtenir le liste de rating d'un produit a prtir son ProductId
def productRat(productId):
    ratings ={}
    for prdId, ligne in product_user_mat.iterrows():
        ratings[prdId] = ligne.values
    return list(ratings[productId])

In [18]:
productRat(184)

[1.0, 0.0, 1.0, 0.0, 0.0, 1.0]

# Application de L'algorithme

In [19]:
def cosine_sim(X, Y):
    c=0; rx=0; ry=0
    if len(X) == len(Y):
        for i in range(len(X)):
            c += X[i]*Y[i] 
            rx += X[i]**2
            ry += Y[i]**2
        return c / ((rx*ry)**0.5)

In [20]:
def make_recommendation(n_prdtRec, userId):
    """
    Parameters
    ----------
    n_productsRecommended  : Nombre de produits à recommander
    k_neighbors            : Nombre de voisins utilisés pour le classement par défaut 40
    userId                 : L'utilisateur pour lequel on va commander les articles
    listProduct            : La liste des produits
    product_user_mat       : Matrice d'evaluation User-Product

    Return
    ------
    liste des n produit à recommander
    """
    R=[]
    for noratedPrdtId in norate[userId]:
        S1=0; S2=0
        for ratedPrdtId in rate[userId]:
            I=productRat(noratedPrdtId);  J=productRat(ratedPrdtId); r=productRatByUser(userId, ratedPrdtId)
            S1 += cosine_sim(I, J)*r
            S2 += cosine_sim(I, J)
        if S2 == 0:
            ri = 0
        else:
            ri = S1 / S2
        R.append((noratedPrdtId, productTitle(noratedPrdtId), ri))
    R = sorted(R, key=lambda x: x[2])
    
    print(f"Les {n_prdtRec} produits recommandé  pour l'utilisateur {userId} :")
    for prdId, title, rating in R[:n_prdtRec]:
        print(f'{prdId} : {title} avec un rating = {rating}')

In [21]:
make_recommendation(
    n_prdtRec = 10,
    userId = 1
)

Les 10 produits recommandé  pour l'utilisateur 1 :
485 : Baby Girl Cartoon Unicorn & Star Print Jumpsuit avec un rating = 2.9340061868334373
477 : Baby Girl Floral Print Bodysuit & Headband avec un rating = 2.9340422196998257
602 : SHEIN Girls Knot Strap Split Hem Butterfly Pattern Velvet Slip Dress avec un rating = 2.9346554235478823
609 : SHEIN Girls Paperbag Waist Corduroy Pants avec un rating = 2.9346554235478823
617 : SHEIN Girls Drop Shoulder Plaid Sweatshirt avec un rating = 2.9346554235478823
619 : SHEIN Girls Solid Cami Top & Belted Pants Set avec un rating = 2.9346554235478823
501 : Baby Girl Contrast Lace Criss Cross Back Bodysuit avec un rating = 2.934764403782554
542 : SHEIN Men Drawstring Waist Rainbow Striped Tape Side Sweatpants avec un rating = 2.935501023332718
560 : Men Zipper Detail Flap Pocket Cargo Pants avec un rating = 2.935501023332718
584 : SHEIN Girls High Neck Flare Dress avec un rating = 2.935501023332718
